In [1]:
import ee
import geemap
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import numpy as np
import ipynb
import time

In [2]:
# Get authentication credentials to initialize ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AWtgzh7Gvb7umhD456xHwoRb6yU1xeOjFe_bHb-5vPPRGoedTkG8Nl3Oum0

Successfully saved authorization token.


In [3]:
# Define geometry for data collection


# Nanda Bet larger S1 extent
# roi = ee.Geometry.Rectangle([69.98568, 22.66953, 72.76651, 24.60271]);
# shp_file = 'data/s1_boundary.shp'
# boundary = geemap.shp_to_ee(shp_file)


# # Nanda Bet smaller region
# roi = ee.Geometry.Rectangle([70.7980547394194417, 23.2880385161501735, 71.5060515087354958, 23.9024834537972986])  
# shp_file = 'data/smaller_region.shp'
# boundary = geemap.shp_to_ee(shp_file)


# # Gujarat extent
# shp_file = 'data/gujarat_extent.shp'
# boundary = geemap.shp_to_ee(shp_file)
# roi = boundary.geometry()


# South Sudan
shp_file = 'data/osm_south_sudan_boundary_v2.shp'
boundary = geemap.shp_to_ee(shp_file)
roi = boundary.geometry()


In [4]:
# Get dem in roi

dem_col = ee.ImageCollection("projects/sat-io/open-datasets/GLO-30").filterBounds(roi)

In [5]:
# Display metadata

print(dem_col.first().getInfo())

{'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [3600, 3600], 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, 29.999861111111112, 0, -0.0002777777777777778, 4.0001388888888885]}], 'version': 1644308331431283, 'id': 'projects/sat-io/open-datasets/GLO-30/Copernicus_DSM_COG_10_N03_00_E030_00_DEM', 'properties': {'num_bands': 1, 'id_no': 'Copernicus_DSM_COG_10_N03_00_E030_00_DEM', 'ysize': 3600, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[29.999965141390838, 3.0000016829220923], [30.00000006847977, 2.9999994935369383], [30.499861137770377, 2.9999995305740357], [30.999722165715863, 2.999999495373395], [30.99985063481075, 3.000112567308768], [30.999992143185963, 3.0002087554201102], [31.00000089447152, 3.000254463118545], [31.000001157010075, 3.999999977649988], [30.99988782781346, 4.00012821797], [30.99979147603467, 4.000269561785379], [30.999745619212014, 4.000278250227774], [30.499861137770385

In [6]:
# Print number of images in image collection

print('Number of images in dem collection:', dem_col.size().getInfo())

Number of images in dem collection: 441


In [7]:
# Merge collection to image by taking mean and clip to boundary

dem = ee.Image(dem_col.mean()).clip(boundary.geometry())

In [17]:
# Define visualization parameters

palette = ['#000000', '#1E5BC0', '#0882E9', '#00A6D7', '#00C67B', '#31D36D', '#69E079',
           '#A5EE87', '#E5FC93', '#F3EF92', '#D1C680', '#B29E70', '#967860', '#88655D',
           '#A68A85', '#C2B1AF', '#E1D8D7', '#FFFFFF'];


dem_vis = {'min': 500, 'max': 3000, 'palette': palette};

In [18]:
# Create map and add layers

M = geemap.Map()
M.centerObject(roi, 6)
M.addLayer(dem,dem_vis,'Elevation')
M

Map(center=[7.372388511008682, 30.179029749002392], controls=(WidgetControl(options=['position', 'transparent_…

In [19]:
# Export dem geotiff to google drive

txt = 'copernicus_dem_180m_ssudan'
task = ee.batch.Export.image.toDrive(image=dem,
                                    description=txt,
                                    scale=180,
                                    region=boundary.geometry(),
                                    fileNamePrefix=txt,
                                    crs='EPSG:4326',
                                    fileFormat='GeoTIFF',
                                    maxPixels=2e10)
task.start()    

In [22]:
# Check task progress

task.list()

[<Task ESUYUTB2V4V4T7DL3P2AQAL6 EXPORT_IMAGE: copernicus_dem_180m_ssudan (RUNNING)>,
 <Task TQVBNT7GYYIRPAPG4CH4U6A5 EXPORT_IMAGE: s1_nandabet_03-03-2020_20m (COMPLETED)>,
 <Task OWIHZ7CGYJ4US6N2SN2XADKO EXPORT_IMAGE: dw_label_nandabet_20200301_20m (COMPLETED)>,
 <Task EHSTRGBKUSWYUIKY4B5QVZFX EXPORT_IMAGE: dw_bands_nandabet_20200301_20m (COMPLETED)>,
 <Task BJGCCBABWUWE37IK5D7VORSB EXPORT_IMAGE: s2_nandabet_20200303_all_bands (COMPLETED)>,
 <Task BGORK4R5DYOO4W7PH2HGWYKW EXPORT_IMAGE: s1_nanda_bet_08-18-2020 (COMPLETED)>,
 <Task XO2LXBBZWYNR6W3ITQF5QMHD EXPORT_IMAGE: s1_everglades_12-06-2021 (COMPLETED)>,
 <Task OPNHMXYRWGANN2IFFQVHRK4Q EXPORT_IMAGE: s1_everglades_12-06-2021 (COMPLETED)>]